In [ ]:
import json

In [ ]:
population = json.load(open("example_problems/circle_packing/outputs/2025-06-22_00-13-25_circle_packing/population.json"))

In [ ]:
# Find the fittest organism in the population
fittest = max(population, key=lambda org: org['evaluation']['fitness'])
print(f"Fittest organism ID: {fittest['id']}")
print(f"Fittest organism fitness: {fittest['evaluation']['fitness']:.8f}")

# Create a mapping of organism ID to organism for quick lookup
org_map = {org['id']: org for org in population}

# Trace ancestry starting from the fittest
def trace_ancestry(organism_id):
    """Trace the ancestry of an organism back to its origins"""
    ancestry = []
    current_id = organism_id
    
    while current_id is not None:
        if current_id in org_map:
            current_org = org_map[current_id]
            ancestry.append({
                'id': current_org['id'],
                'fitness': current_org['evaluation']['fitness'],
                'parent_id': current_org.get('parent_id')
            })
            current_id = current_org.get('parent_id')
        else:
            break
    
    return ancestry

# Get ancestry of the fittest organism
ancestry = trace_ancestry(fittest['id'])

# Calculate the best fitness at each point in time (up to each ancestor)
# We need to determine what the best fitness was when each ancestor was alive
def calculate_best_at_time(ancestry):
    """Calculate what the best fitness was at the time each ancestor existed"""
    # Sort all organisms by ID (assuming ID is chronological)
    all_orgs_sorted = sorted(population, key=lambda org: org['id'])
    
    results = []
    for ancestor in ancestry:
        # Find the best fitness among all organisms with ID <= ancestor's ID
        best_at_time = max(
            (org['evaluation']['fitness'] for org in all_orgs_sorted if org['id'] <= ancestor['id']),
            default=ancestor['fitness']
        )
        
        # Check if this ancestor was the best when it was created
        # Find the best fitness among organisms with ID < ancestor's ID
        best_before = max(
            (org['evaluation']['fitness'] for org in all_orgs_sorted if org['id'] < ancestor['id']),
            default=0.0
        )
        was_best = ancestor['fitness'] > best_before
        
        results.append({
            **ancestor,
            'best_at_time': best_at_time,
            'was_best_when_created': was_best
        })
    return results

ancestry_with_best = calculate_best_at_time(ancestry)

print(f"\nAncestry of fittest organism (from fittest to oldest ancestor):")
print("=" * 85)
print(f"{'#':>2s} | {'ID':>8s} | {'Fitness':>12s} | {'Best at Time':>12s} | {'Parent':>8s}")
print("=" * 85)
for i, ancestor in enumerate(ancestry_with_best):
    parent_str = str(ancestor['parent_id']) if ancestor['parent_id'] is not None else 'None'
    id_str = f"{ancestor['id']}*" if ancestor['was_best_when_created'] else str(ancestor['id'])
    print(f"{i+1:2d} | {id_str:>8s} | {ancestor['fitness']:12.8f} | {ancestor['best_at_time']:12.8f} | {parent_str:>8s}")

In [ ]:
# Find all organisms that were the best so far at any point
def find_best_so_far_organisms():
    """Find all organisms that were the best fitness when they were created"""
    # Sort all organisms by ID (chronological order)
    all_orgs_sorted = sorted(population, key=lambda org: org['id'])
    
    best_so_far_orgs = []
    current_best_fitness = 0.0
    
    for org in all_orgs_sorted:
        fitness = org['evaluation']['fitness']
        if fitness > current_best_fitness:
            best_so_far_orgs.append(org)
            current_best_fitness = fitness
    
    return best_so_far_orgs

best_orgs = find_best_so_far_organisms()
print(f"Number of organisms that were best so far at any point: {len(best_orgs)}")
print("\nThese organisms were:")
for org in best_orgs:
    print(f"  ID {org['id']:4d}: fitness {org['evaluation']['fitness']:.8f}")

# Create graphviz visualization of all their ancestries
import graphviz

def get_all_ancestors(organism_ids):
    """Get all ancestors for a set of organism IDs"""
    all_ancestors = set()
    
    for org_id in organism_ids:
        ancestry = trace_ancestry(org_id)
        for ancestor in ancestry:
            all_ancestors.add(ancestor['id'])
    
    return all_ancestors

def create_ancestry_graph(organism_ids):
    """Create a directed graph showing ancestry relationships"""
    dot = graphviz.Digraph(comment='Evolution Ancestry')
    dot.attr(rankdir='TB')  # Top to bottom layout
    dot.attr('node', shape='box', style='rounded,filled')
    
    # Get all relevant organisms (ancestors of best-so-far organisms)
    relevant_org_ids = get_all_ancestors(organism_ids)
    
    # Add nodes
    best_org_ids = set(org['id'] for org in best_orgs)
    
    for org_id in relevant_org_ids:
        if org_id in org_map:
            org = org_map[org_id]
            fitness = org['evaluation']['fitness']
            
            # Different colors for best-so-far vs ancestors
            if org_id in best_org_ids:
                color = 'lightblue'
                label = f"ID: {org_id}*\\nFitness: {fitness:.6f}"
            else:
                color = 'lightgray'
                label = f"ID: {org_id}\\nFitness: {fitness:.6f}"
            
            dot.node(str(org_id), label, fillcolor=color)
    
    # Add edges (parent -> child relationships)
    for org_id in relevant_org_ids:
        if org_id in org_map:
            org = org_map[org_id]
            parent_id = org.get('parent_id')
            if parent_id is not None and parent_id in relevant_org_ids:
                dot.edge(str(parent_id), str(org_id))
    
    return dot

# Create the graph
best_org_ids = [org['id'] for org in best_orgs]
graph = create_ancestry_graph(best_org_ids)

# Display the graph
graph

In [ ]:
# Create markdown documentation of the best organism's ancestry
def create_best_ancestry_markdown():
    # Get the ancestry data we calculated earlier
    ancestry_with_best = calculate_best_at_time(ancestry)
    
    # Start building the markdown content
    md_content = []
    md_content.append("# Best Organism Ancestry Analysis")
    md_content.append("")
    md_content.append(f"This document traces the complete ancestry of the fittest organism (ID: {fittest['id']}) with fitness {fittest['evaluation']['fitness']:.8f}.")
    md_content.append("")
    md_content.append("Each section shows an ancestor in the lineage, from the fittest organism back to the original ancestor.")
    md_content.append("Organisms marked with * were the best fitness when they were created.")
    md_content.append("")
    md_content.append("---")
    md_content.append("")
    
    # Go through each ancestor
    for i, ancestor in enumerate(ancestry_with_best):
        # Add header for this ancestor
        was_best_marker = "*" if ancestor['was_best_when_created'] else ""
        md_content.append(f"## Ancestor #{i+1}: Organism {ancestor['id']}{was_best_marker}")
        md_content.append("")
        
        # Add the info table
        md_content.append("| Property | Value |")
        md_content.append("|----------|-------|")
        md_content.append(f"| **ID** | {ancestor['id']}{was_best_marker} |")
        md_content.append(f"| **Fitness** | {ancestor['fitness']:.8f} |")
        md_content.append(f"| **Best at Time** | {ancestor['best_at_time']:.8f} |")
        parent_display = ancestor['parent_id'] if ancestor['parent_id'] is not None else 'None'
        md_content.append(f"| **Parent ID** | {parent_display} |")
        md_content.append(f"| **Was Best When Created** | {'Yes' if ancestor['was_best_when_created'] else 'No'} |")
        md_content.append("")
        
        # Add the solution code
        if ancestor['id'] in org_map:
            solution = org_map[ancestor['id']]['solution']
            md_content.append("### Solution Code")
            md_content.append("")
            md_content.append("```python")
            md_content.append(solution)
            md_content.append("```")
            md_content.append("")
        
        md_content.append("---")
        md_content.append("")
    
    # Join all content and write to file
    full_content = "\n".join(md_content)
    
    with open("best_ancestry.md", "w") as f:
        f.write(full_content)
    
    print(f"Created best_ancestry.md with {len(ancestry_with_best)} ancestors documented")
    print(f"Total file size: {len(full_content)} characters")

# Run the function
create_best_ancestry_markdown()